In [1]:
import os
from pathlib import Path

root_directory = "/cluster/home/wueestm/f3loc/Depth-Anything-V2"
os.chdir(root_directory)
root_directory
Path.cwd()

PosixPath('/cluster/home/wueestm/f3loc/Depth-Anything-V2')

In [2]:
import numpy as np

In [3]:
import cv2
import torch

from depth_anything_v2.dpt import DepthAnythingV2

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]}
}

encoder = 'vitl' # 'vitb' #or 'vits', 'vitb'
dataset = 'hypersim' # 'hypersim' for indoor model, 'vkitti' for outdoor model
# max_depth = 20 # 20 for indoor model, 80 for outdoor model

#model = DepthAnythingV2(**{**model_configs[encoder], 'max_depth': max_depth})
model = DepthAnythingV2(**model_configs[encoder])
model.load_state_dict(torch.load(f'checkpoints/depth_anything_v2_metric_{dataset}_{encoder}.pth', map_location='cpu'))
model.eval()


/cluster/home/wueestm/anaconda3/envs/f3loc/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
xFormers not available
xFormers not available


DepthAnythingV2(
  (pretrained): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-23): 24 x NestedTensorBlock(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (drop): Dropout(p=0.0, inplace=Fal

In [6]:
# Load your raw image
raw_image = raw_img
input_tensor, _ =  model.image2tensor(raw_image)#, input_size=1440/16*14)

In [18]:
# Forward pass through the DINO model to get embeddings
with torch.no_grad():
    features_all = model.pretrained.get_intermediate_layers(input_tensor, model.intermediate_layer_idx[model.encoder], reshape=True)

In [22]:
features_all[3].shape

torch.Size([1, 1024, 49, 37])

In [10]:
model.encoder

'vitl'

In [12]:
model.intermediate_layer_idx[model.encoder][-1]

23

In [13]:
# Forward pass through the DINO model to get embeddings
with torch.no_grad():
    features = model.pretrained.get_intermediate_layers(input_tensor, reshape=True)

In [25]:
import torch.nn.functional as F
resized_features = F.interpolate(features[0], size=(120, 90), mode='bilinear', align_corners=True)

In [27]:
features[0].shape

torch.Size([1, 1024, 49, 37])

In [26]:
resized_features.shape

torch.Size([1, 1024, 120, 90])

In [28]:
_, _, h, w = resized_features.shape

In [31]:
1440/16

90.0

In [33]:
1440//16

90

In [34]:
h

120

In [ ]:
# Forward pass through the DINO model to get embeddings
with torch.no_grad():
    features = model.pretrained.get_intermediate_layers(input_tensor, model.intermediate_layer_idx[model.encoder], reshape=True)

In [53]:
features[-1].shape

torch.Size([1, 1024, 49, 37])

In [55]:
features[0].shape

torch.Size([1, 1024, 49, 37])

In [56]:
features[1].shape

torch.Size([1, 1024, 49, 37])

In [57]:
features[2].shape

torch.Size([1, 1024, 49, 37])

In [58]:
features[3].shape

torch.Size([1, 1024, 49, 37])

In [60]:
features[4].shape

IndexError: tuple index out of range

In [59]:
features[0][0][0].shape

torch.Size([49, 37])

In [39]:
49*14

686

In [40]:
37*14

518

In [44]:
49/37

1.3243243243243243

In [43]:
raw_img.shape[0]/raw_img.shape[1]

1.3333333333333333

In [45]:
raw_img.shape[0]/49

39.183673469387756

In [36]:
features[0][0][0].shape

torch.Size([1813, 1024])

In [33]:
features[1]

(tensor([[[-6.0077e-01,  2.1814e+00,  8.9524e-01,  ...,  5.4303e-01,
           -3.8767e-02, -4.4782e+00],
          [-1.7773e+00,  9.4267e-01, -4.0352e-03,  ...,  3.1797e-02,
           -4.2511e-01,  3.3014e+00],
          [-2.5500e-01,  1.5134e-02,  4.5019e-01,  ..., -1.4100e+00,
            7.5691e-01,  6.6905e+00],
          ...,
          [ 1.3716e+00, -9.6508e-01,  6.5209e-03,  ...,  8.0838e-01,
            6.3469e-01, -3.3081e+00],
          [ 1.1796e+00, -5.3309e-01, -8.7790e-01,  ...,  2.0329e+00,
            1.5308e-01, -1.2342e+00],
          [ 8.1156e-01, -1.9211e+00, -4.4640e-01,  ...,  2.3958e+00,
            8.6098e-01, -5.4680e+00]]]),
 tensor([[ 0.1375,  0.4776,  1.0067,  ...,  0.2970, -0.4340, -0.0955]]))

In [24]:
model.intermediate_layer_idx[model.encoder]

[4, 11, 17, 23]

In [26]:
input_tensor

(tensor([[[[ 1.2214,  1.2214,  1.2200,  ..., -0.1271, -0.1314, -0.1314],
           [ 1.2375,  1.2375,  1.2374,  ..., -0.1285, -0.1314, -0.1314],
           [ 1.2573,  1.2573,  1.2588,  ..., -0.1282, -0.1332, -0.1313],
           ...,
           [-0.7361, -0.7926, -0.7160,  ...,  0.1769,  0.1933,  0.2145],
           [-0.6499, -0.7348, -0.7767,  ...,  0.1737,  0.1949,  0.2111],
           [-0.6457, -0.6694, -0.7334,  ...,  0.1785,  0.1949,  0.2111]],
 
          [[ 0.9580,  0.9580,  0.9565,  ..., -0.4032, -0.4076, -0.4076],
           [ 0.9744,  0.9744,  0.9743,  ..., -0.4045, -0.4076, -0.4076],
           [ 0.9946,  0.9946,  0.9962,  ..., -0.4043, -0.4094, -0.4075],
           ...,
           [-0.7981, -0.8559, -0.7776,  ...,  0.0302,  0.0470,  0.0686],
           [-0.7100, -0.7968, -0.8397,  ...,  0.0270,  0.0486,  0.0651],
           [-0.7058, -0.7300, -0.7953,  ...,  0.0319,  0.0486,  0.0651]],
 
          [[ 0.5834,  0.5834,  0.5819,  ..., -0.7718, -0.7761, -0.7761],
           [ 

In [19]:
import cv2
import torch
import numpy as np
from torchvision.transforms import Compose

# Assuming DepthAnythingV2 is already defined as above
# Instantiate your model
depth_model = DepthAnythingV2()  # Add necessary parameters here if required

# Load your raw image (example: raw_image = np.random.rand(1920, 1440, 3) * 255)
raw_image = raw_img

# Prepare the image tensor
input_tensor = DepthAnythingV2.image_to_tensor(raw_image)

# Forward pass through the DINO model to get embeddings
with torch.no_grad():
    features = depth_model.pretrained.get_intermediate_layers(input_tensor, depth_model.intermediate_layer_idx[depth_model.encoder], return_class_token=True)

# Extract the final embeddings from the features
final_embeddings = features[-1]  # Assuming the last layer corresponds to final embeddings
print(final_embeddings.shape)  # This will give you the shape of the embeddings


AttributeError: type object 'DepthAnythingV2' has no attribute 'image_to_tensor'

In [5]:
import time

image_path_ls = [
    "/cluster/home/wueestm/f3loc/metric3d/data/hge_customized_complete/non-aligned/rgb/00000-0.png",
    "/cluster/home/wueestm/f3loc/metric3d/data/hge_customized_complete/non-aligned/rgb/00120-0.png",
    "/cluster/home/wueestm/f3loc/metric3d/data/hge_customized_complete/non-aligned/rgb/00090-0.png",
    "/cluster/home/wueestm/f3loc/metric3d/data/hge_customized_complete/non-aligned/rgb/00171-0.jpg"
]

for image_path in image_path_ls:
    #image_path = "/cluster/home/wueestm/f3loc/metric3d/data/hge_customized_complete/non-aligned/rgb/"
    raw_img = cv2.imread(image_path)
    start_time = time.time()
    #depth = model.infer_image(raw_img) # HxW depth map in meters in numpy
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time taken: {elapsed_time:.4f} seconds")

Time taken: 0.0000 seconds


In [5]:
model.pretrained

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-23): 24 x NestedTensorBlock(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )


In [9]:
import torch
import torch.nn.functional as F

# Initialize your DepthAnythingV2 model (using DINOv2 'vitl' for example)
model = DepthAnythingV2(encoder='vitl')  # Adjust encoder if necessary

print("model initialized")

# Ensure the model is on the appropriate device (CPU, CUDA, or MPS)
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
model = model.to(DEVICE)

# Assuming raw_image is the (1920, 1440, 3) image (loaded as a NumPy array)
raw_image = raw_img  # Your image here

# Step 1: Preprocess the image using the image2tensor method (already part of the DepthAnythingV2 class)
input_tensor, (h, w) = model.image2tensor(raw_image)

print("image preprocessed")

# Step 2: Pass the image through the pretrained DINO model to get final layer features
with torch.no_grad():
    # Get the patch size based on input image
    patch_h, patch_w = input_tensor.shape[-2] // 14, input_tensor.shape[-1] // 14
    
    # Extract features from the final layer of the DINO transformer
    # Here, we use the final index for the transformer based on the encoder type ('vitl' in this case)
    print("before final_layer_idx")
    final_layer_idx = [model.intermediate_layer_idx[model.encoder][-1]]  # Only take the final layer index
    
    # Get the output from the final layer
    print("before final features")
    final_features = model.pretrained.get_intermediate_layers(
        input_tensor, 
        final_layer_idx, 
        return_class_token=True  # Return both patch tokens and class token
    )

# The variable `final_features` now holds the final layer output of the DINO transformer.
# You can process it further based on your application (e.g., classification, feature extraction, etc.)


model initialized
image preprocessed
before final_layer_idx
before final features


In [18]:
import torch

# Assuming your DepthAnythingV2 is already instantiated
model = DepthAnythingV2(encoder='vitl')

# Send model to appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("model initialized")

# Your input image of shape (1920, 1440, 3)
raw_image = raw_img  # Provide your image here (e.g., as a numpy array or tensor)

# Step 1: Preprocess the image
input_tensor, (h, w) = model.image2tensor(raw_image)
input_tensor = input_tensor.unsqueeze(0)  # Add batch dimension

print("image preprocessed")

# Step 2: Pass the input tensor through the pretrained DinoVisionTransformer
with torch.no_grad():
    # Only pass through the 'pretrained' (DinoVisionTransformer) part of the model
    print("before final features")
    transformer_output = model.pretrained(input_tensor.to(device))

# Step 3: Extract the final embeddings
# The output of the pretrained DINO transformer is typically a tuple where the first element is the patch embeddings
final_embeddings = transformer_output

print(f"Shape of final embeddings: {final_embeddings.shape}")


model initialized
image preprocessed
before final features


ValueError: too many values to unpack (expected 4)

In [16]:
model.encoder

'vitl'

In [14]:
type(final_features[0][0])

torch.Tensor

In [15]:
final_features[0][0].shape

torch.Size([1, 1813, 1024])

In [29]:
type(raw_img)

numpy.ndarray

In [30]:
type(depth)

numpy.ndarray

In [6]:
raw_img.shape

(1920, 1440, 3)

In [36]:
depth.max()

4.0938826

In [37]:
depth.min()

0.0

#### Fine tune model

In [4]:
print(model)

DepthAnythingV2(
  (pretrained): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-23): 24 x NestedTensorBlock(
        (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=1024, out_features=3072, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=1024, out_features=1024, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (drop): Dropout(p=0.0, inplace=Fal

In [5]:
list(model.children())

[DinoVisionTransformer(
   (patch_embed): PatchEmbed(
     (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
     (norm): Identity()
   )
   (blocks): ModuleList(
     (0-23): 24 x NestedTensorBlock(
       (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
       (attn): MemEffAttention(
         (qkv): Linear(in_features=1024, out_features=3072, bias=True)
         (attn_drop): Dropout(p=0.0, inplace=False)
         (proj): Linear(in_features=1024, out_features=1024, bias=True)
         (proj_drop): Dropout(p=0.0, inplace=False)
       )
       (ls1): LayerScale()
       (drop_path1): Identity()
       (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
       (mlp): Mlp(
         (fc1): Linear(in_features=1024, out_features=4096, bias=True)
         (act): GELU(approximate='none')
         (fc2): Linear(in_features=4096, out_features=1024, bias=True)
         (drop): Dropout(p=0.0, inplace=False)
       )
       (ls2): LayerScale()
       (drop_

In [6]:
list(model.children())[0]

DinoVisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0-23): 24 x NestedTensorBlock(
      (norm1): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (attn): MemEffAttention(
        (qkv): Linear(in_features=1024, out_features=3072, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=1024, out_features=1024, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): LayerScale()
      (drop_path1): Identity()
      (norm2): LayerNorm((1024,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
      (ls2): LayerScale()
      (drop_path2): Identity()
    )


In [15]:
list(model.children())[1]